1. Попередня обробка тексту для RNN:
  * Завантажте текстовий корпус та перетворіть його у послідовність символів або слів.
  * Створіть словник для перетворення символів або слів у числові індекси.
  * Розділіть текст на короткі фрагменти (наприклад, послідовності по 100 символів або слів), які будуть використовуватися як вхідні дані для RNN.

2. Створення архітектури RNN:
  * Визначте рекурентну нейронну мережу, використовуючи шари LSTM або GRU (в залежності від обраної бібліотеки).
  * Додайте один або більше рекурентних шарів з вибраною кількістю нейронів у кожному шарі.
  * Додайте повнозв’язний шар із softmax для прогнозування ймовірностей наступних символів або слів.

Ідея в примінені RNN GRU для генерації новин з різних джерел. Я всяв с Kaggle датасет з новинами. Зробив модель для кожного джерела.

In [1]:
import  torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import numpy as np
from tqdm import tqdm
import random

In [2]:
# Завантажте текстовий корпус та перетворіть його у послідовність символів або слів.
df = pd.read_csv('data/news.csv', index_col=0)
df=df.dropna()
grouped_texts = df.groupby('group_name')['text'].apply(list).to_dict()
print(grouped_texts.keys())

dict_keys(['BBC Russia', 'Meduza', 'Novaya Gazeta', 'RBK', 'RIA News', 'Shocked Ukraine', 'TASS', 'The New York Times', 'Ukraine Now', 'Ukraine24', 'Washington Post', 'УНИАН'])


In [3]:
# Check if MPS is available
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [4]:
# Create vocabulary

# Create vocabulary
def build_vocab(df, min_freq=2):
    counter = Counter()
    for text in df:
        counter.update(text.split())
    vocab = {word: idx + 1 for idx, (word, count) in enumerate(counter.items()) if count >= min_freq}
    vocab['<PAD>'] = 0
    vocab['<UNK>'] = len(vocab)
    return vocab

# Custom Dataset
class TweetDataset(Dataset):
    def __init__(self, texts, vocab, seq_length=30):
        self.vocab = vocab
        self.seq_length = seq_length
        self.word_to_idx = vocab
        self.idx_to_word = {idx: word for word, idx in vocab.items()}
        self.data = self.prepare_sequences(texts)
        
    def prepare_sequences(self, texts):        
        sequences = []
        for text in texts:
            tokens = text.split()
            idx_sequence = [self.word_to_idx.get(word, self.word_to_idx['<UNK>']) for word in tokens]
            sequences.append(idx_sequence)
        return sequences
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sequence = self.data[idx]
        if len(sequence) > self.seq_length:
            start_idx = np.random.randint(0, len(sequence) - self.seq_length)
            sequence = sequence[start_idx:start_idx + self.seq_length]
        else:
            sequence = sequence + [self.word_to_idx['<PAD>']] * (self.seq_length - len(sequence))
        
        x = torch.tensor(sequence[:-1])
        y = torch.tensor(sequence[1:])
        return x, y


In [5]:
# GRU Model (more memory efficient than LSTM)
class TweetGenerator(nn.Module):
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=256):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, x):
        embed = self.embedding(x)
        gru_out, _ = self.gru(embed)
        return self.fc(gru_out)

In [7]:
def train(model,dataloader,optimizer,criterion,vocab,num_epochs):
    # Training loop    
    for epoch in tqdm(range(num_epochs)):
        for batch_x, batch_y in dataloader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            
            optimizer.zero_grad()
            output = model(batch_x)
            loss = criterion(output.reshape(-1, len(vocab)), batch_y.reshape(-1))
            loss.backward()
            optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')
    return model

    # Generate text
def generate_tweet(model, vocab, seed_text="hello", max_length=30):
    model.eval()
    word_to_idx = vocab
    idx_to_word = {idx: word for word, idx in vocab.items()}
    
    words = seed_text.split()
    current_ids = torch.tensor([word_to_idx.get(w, word_to_idx['<UNK>']) for w in words]).unsqueeze(0).to(device)
    
    with torch.no_grad():
        for _ in range(max_length - len(words)):
            output = model(current_ids)
            next_word_idx = torch.argmax(output[0, -1]).item()
            words.append(idx_to_word[next_word_idx])
            current_ids = torch.cat([current_ids, torch.tensor([[next_word_idx]]).to(device)], dim=1)
            
    return ' '.join(words)

In [10]:
models = {}
vocabs = {}
num_epochs=300
n_tweets = 1000

selected_sources = ['BBC Russia','УНИАН','Ukraine24']
for source,text in grouped_texts.items():
    if source not in selected_sources:
        continue
    print(source)
    # print(f'Size of train data: {len(text)}')
    # text = random.sample(text, k=n_tweets) 
    text = text[:n_tweets]

    vocab = build_vocab(text)
    vocabs[source]=vocab #if do one vocab to all model, then i got memory error
    dataset = TweetDataset(text, vocab)
    dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

    model = TweetGenerator(len(vocab)).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())

    models[source]=train(model=model, 
                        dataloader=dataloader, 
                        optimizer=optimizer,
                        criterion=criterion, 
                        num_epochs=num_epochs,
                        vocab=vocab)
    
    print(generate_tweet(models[source], vocabs[source], seed_text="Украина"))
    print(generate_tweet(models[source], vocabs[source], seed_text="Зеленский"))
    print(generate_tweet(models[source], vocabs[source], seed_text="Путин"))

BBC Russia


100%|██████████| 300/300 [18:31<00:00,  3.71s/it]


Epoch 300, Loss: 0.1788
Украина будет создавать для России постоянно <UNK> и абсолютно неприемлемую <UNK> - сказал Путин в четверг в телеобращении в котором <UNK> объявил о начале военных действий против <UNK> <UNK> 2021
Зеленский – <UNK> <UNK> с нашей <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Путин о начале "специальной военной операции" в Донбассе. Главное <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Ukraine24


100%|██████████| 300/300 [16:56<00:00,  3.39s/it]


Epoch 300, Loss: 0.1327
Украина <UNK> <UNK> <UNK> <UNK> <UNK> - <UNK> в <UNK> <UNK> <UNK> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Зеленский созвонился с <UNK> Украину в <UNK> но и в <UNK> <UNK> <UNK> страну сделали <UNK> <UNK> <UNK> с <UNK> <UNK> <UNK> <UNK> цвета <UNK> <UNK> <UNK> в этом <UNK>
Путин заявил, что начинает военную операцию в Украине. Он <UNK> что в его <UNK> нет оккупации - а только <UNK> и <UNK> . Около 5 утра в районе аэропорта <UNK>
УНИАН


100%|██████████| 300/300 [17:32<00:00,  3.51s/it]


Epoch 300, Loss: 0.2095
Украина была и остается <UNK> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Зеленский сообщает, что провел переговоры с <UNK> <UNK> и <UNK> <UNK> <UNK> что <UNK> <UNK> <UNK> <UNK> санкции и <UNK> и военная поддержка <UNK> должен принудить РФ к миру" <PAD>
Путин готов <UNK> в <UNK> с <UNK> вы <UNK> на <UNK> <UNK> й <UNK> <UNK> <UNK> с <UNK> що <UNK> <UNK> в <UNK> області у місті <UNK> ВЧ <UNK> <UNK>
